In [ ]:

import xml.etree.ElementTree as ET
import os
from os import listdir
from os.path import join
 
classes = ['嗜中性-带形核', '中幼红细胞', '嗜中性-中幼粒细胞', '嗜中性-晚幼粒细胞', '嗜中性-分叶核', '原始细胞', '嗜酸', '晚幼红细胞', '早幼粒细胞', 
'成熟淋巴细胞', '单核细胞', '原红细胞', '浆细胞', '早幼红细胞', '嗜碱', '异常淋巴细胞', '粒细胞系统-嗜中性-核浆发育失衡', '幼淋巴细胞']

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)
 
def convert_annotation(image_id):
    in_file = open('Annotations/%s.xml'%(image_id), encoding = 'utf-8')
    out_file = open('labels/%s.txt'%(image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            print(cls)
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
 
val_percent = 0#测试集占总数据集的比例，默认0.1，如果测试集和训练集已经划分开了，则修改相应代码
data_path = ''#在darknet文件夹的相对路径，见github的说明，根据自己需要修改，注意此处也可以用绝对路径
if not os.path.exists('labels/'):
    os.makedirs('labels/')
image_ids = [f for f in os.listdir('Annotations')]#存放XML数据的文件夹
train_file = open('train.txt', 'w')
for i, image_id in enumerate(image_ids):
    if image_id[-3:] == "xml":#有些时候jpg和xml文件是放在同一文件夹下的，所以要判断一下后缀
        train_file.write(data_path + '%s\n'%(image_id[:-3] + 'jpg'))
    convert_annotation(image_id[:-4])
train_file.close()


In [27]:
import os
import random
import shutil
from shutil import move,copy2
import numpy as np

path = "data/"
img_path = os.path.join(path,"images")
label_path = os.path.join(path,"labels")
#print(img_path,label_path)
img_list = os.listdir(img_path)
label_list = os.listdir(label_path)
num = len(img_list)
index = list(range(num))
random.shuffle(index)
print(index)


train_dir = os.path.join(path,"train")
valid_dir = os.path.join(path,"valid")
test_dir = os.path.join(path,"test")

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    if not os.path.exists(train_dir+"/images"):
        os.makedirs(train_dir+"/images")
    if not os.path.exists(train_dir+"/labels"):
        os.makedirs(train_dir+"/labels")

if not os.path.exists(valid_dir):
    os.makedirs(valid_dir)
    if not os.path.exists(valid_dir+"/images"):
        os.makedirs(valid_dir+"/images")
    if not os.path.exists(valid_dir+"/labels"):
        os.makedirs(valid_dir+"/labels")

if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    if not os.path.exists(test_dir+"/images"):
        os.makedirs(test_dir+"/images")
    if not os.path.exists(test_dir+"/labels"):
        os.makedirs(test_dir+"/labels")


n = 0
for i in index:
    #i = 0
    name = img_list[i][:-4]
    img_name = name+".jpg"
    label_name = name+".txt"
    print(label_name)
    print(img_name)
    filename_img = os.path.join(img_path,img_name)
    filename_label = os.path.join(label_path,label_name)
    print(filename_img)
    print(filename_label)
    print("\n")

    if n <= num * 0.6:
        copy2(filename_img,os.path.join(train_dir,"images"))
        copy2(filename_label,os.path.join(train_dir,"labels"))
    elif n > num * 0.6 and n < num * 0.8:
        copy2(filename_img,os.path.join(valid_dir,"images"))
        copy2(filename_label,os.path.join(valid_dir,"labels"))
    else:
        copy2(filename_img,os.path.join(test_dir,"images"))
        copy2(filename_label,os.path.join(test_dir,"labels"))
    n+=1
    if n == 3:
        break


[12, 5, 0, 10, 2, 14, 1, 11, 13, 3, 6, 7, 8, 4, 9]
0_Concern-In-China-As-Mystery-Virus-Spreads_jpg.rf.3135dfc5feab288d76a4ccfd22dfc5bf 

w1240-p16x9-0e48e0098f6e832f27d8b581b33bbc72b9967a63_jpg.rf.34ed1e8f70eebdabaf43ab9d40dc1c9b.txt
w1240-p16x9-0e48e0098f6e832f27d8b581b33bbc72b9967a63_jpg.rf.34ed1e8f70eebdabaf43ab9d40dc1c9b.jpg
data/images\w1240-p16x9-0e48e0098f6e832f27d8b581b33bbc72b9967a63_jpg.rf.34ed1e8f70eebdabaf43ab9d40dc1c9b.jpg
data/labels\w1240-p16x9-0e48e0098f6e832f27d8b581b33bbc72b9967a63_jpg.rf.34ed1e8f70eebdabaf43ab9d40dc1c9b.txt


15391513329330sooq10859_jpg.rf.89c8524c2096175fa2c728e5d73f1c28.txt
15391513329330sooq10859_jpg.rf.89c8524c2096175fa2c728e5d73f1c28.jpg
data/images\15391513329330sooq10859_jpg.rf.89c8524c2096175fa2c728e5d73f1c28.jpg
data/labels\15391513329330sooq10859_jpg.rf.89c8524c2096175fa2c728e5d73f1c28.txt


0_Concern-In-China-As-Mystery-Virus-Spreads_jpg.rf.3135dfc5feab288d76a4ccfd22dfc5bf.txt
0_Concern-In-China-As-Mystery-Virus-Spreads_jpg.rf.3135dfc5feab